In [1]:
from __future__ import (absolute_import, division, print_function, unicode_literals)
import os
import io

from surprise import KNNBaseline, Reader
from surprise import Dataset

import cPickle as pickle
# 重建歌单id到歌单名的映射字典
id_name_dic = pickle.load(open("popular_playlist.pkl","rb"))
print("加载歌单id到歌单名的映射字典完成...")
# 重建歌单名到歌单id的映射字典
name_id_dic = {}
for playlist_id in id_name_dic:
    name_id_dic[id_name_dic[playlist_id]] = playlist_id
print("加载歌单名到歌单id的映射字典完成...")

加载歌单id到歌单名的映射字典完成...
加载歌单名到歌单id的映射字典完成...


In [2]:
file_path = os.path.expanduser('./popular_music_suprise_format.txt')
# 指定文件格式
reader = Reader(line_format='user item rating timestamp', sep=',')
# 从文件读取数据
music_data = Dataset.load_from_file(file_path, reader=reader)
# 计算歌曲和歌曲之间的相似度
print("构建数据集...")
trainset = music_data.build_full_trainset()
#sim_options = {'name': 'pearson_baseline', 'user_based': False}

构建数据集...


In [5]:
############################################################################
id_name_dic.keys()[3]
print(id_name_dic[id_name_dic.keys()[3]])
print(trainset.n_items)
print(trainset.n_users)

♬毕业季，我想大声告诉你♬
50539
1076


## 1.2.1 模板之查找最近的user(在这里是歌单)

In [9]:
###########################################################################
#模板之查找最近的user(在这里是歌单)
print("开始训练模型...")
#sim_options = {'user_based': False}
#algo = KNNBaseline(sim_options=sim_options)
algo = KNNBaseline()
algo.train(trainset)
#
current_playlist = name_id_dic.keys()[19]
print("歌单名称", current_playlist)
#

开始训练模型...
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
歌单名称 如果终究要离开，我会忍住不责怪


In [10]:
# 取出近邻
# 映射名字到id
playlist_id = name_id_dic[current_playlist]
print("歌单id", playlist_id)
# 取出来对应的内部user id => to_inner_uid
playlist_inner_id = algo.trainset.to_inner_uid(playlist_id)
print("内部id", playlist_inner_id)
playlist_neighbors = algo.get_neighbors(playlist_inner_id, k=10)
# 把歌曲id转成歌曲名字
# to_raw_uid映射回去
playlist_neighbors = (algo.trainset.to_raw_uid(inner_id) for inner_id in playlist_neighbors)
playlist_neighbors = (id_name_dic[playlist_id] for playlist_id in playlist_neighbors)

歌单id 448831188
内部id 701


In [11]:
print()
print("和歌单 《", current_playlist, "》 最接近的10个歌单为：\n")
for playlist in playlist_neighbors:
    print(playlist, algo.trainset.to_inner_uid(name_id_dic[playlist]))


和歌单 《 如果终究要离开，我会忍住不责怪 》 最接近的10个歌单为：

当过千评论的华语翻唱遇上“原唱”【更新】 1
【华语】暖心物语 纯白思念 3
〖循环〗单曲循环是强迫症吗？ 4
周杰伦地表最强演唱会2017520南京站曲目 5
大神爱翻唱1：华语篇 8
『华语/回忆』95后陪伴我中学时期的歌曲 13
暗暗作祟| 不甘朋友不敢恋人 15
有没有一首歌让你泪流满面 17
专属你的周杰伦 18
你知道思念一个人的滋味吗 21


### 1.2.2 模板之针对用户进行预测

In [12]:
import cPickle as pickle
# 重建歌曲id到歌曲名的映射字典
song_id_name_dic = pickle.load(open("popular_song.pkl","rb"))
print("加载歌曲id到歌曲名的映射字典完成...")
# 重建歌曲名到歌曲id的映射字典
song_name_id_dic = {}
for song_id in song_id_name_dic:
    song_name_id_dic[song_id_name_dic[song_id]] = song_id
print("加载歌曲名到歌曲id的映射字典完成...")

加载歌曲id到歌曲名的映射字典完成...
加载歌曲名到歌曲id的映射字典完成...


In [13]:
#内部编码的5号用户
user_inner_id = 5
user_rating = trainset.ur[user_inner_id]
items = map(lambda x:x[0], user_rating)
for song in items:
    print(algo.predict(user_inner_id, song, r_ui=1), song_id_name_dic[algo.trainset.to_raw_iid(song)])

user: 5          item: 1017       r_ui = 1.00   est = 1.00   {u'was_impossible': False} 英雄	周杰伦
user: 5          item: 566        r_ui = 1.00   est = 1.00   {u'was_impossible': False} 无双	周杰伦
user: 5          item: 588        r_ui = 1.00   est = 1.00   {u'was_impossible': False} 她的睫毛	周杰伦
user: 5          item: 614        r_ui = 1.00   est = 1.00   {u'was_impossible': False} 开不了口	周杰伦
user: 5          item: 1018       r_ui = 1.00   est = 1.00   {u'was_impossible': False} 床边故事	周杰伦
user: 5          item: 529        r_ui = 1.00   est = 1.00   {u'was_impossible': False} 窃爱	周杰伦
user: 5          item: 624        r_ui = 1.00   est = 1.00   {u'was_impossible': False} 以父之名	周杰伦
user: 5          item: 536        r_ui = 1.00   est = 1.00   {u'was_impossible': False} 美人鱼	周杰伦
user: 5          item: 1019       r_ui = 1.00   est = 1.00   {u'was_impossible': False} 屋顶	温岚
user: 5          item: 1020       r_ui = 1.00   est = 1.00   {u'was_impossible': False} Try	派伟俊
user: 5          item: 1021       r_ui = 

## 2.用矩阵分解进行预测

In [12]:
#2.用矩阵分解进行预测
### 使用NMF
from surprise import NMF, evaluate
from surprise import Dataset

In [13]:
file_path = os.path.expanduser('./popular_music_suprise_format.txt')
# 指定文件格式
reader = Reader(line_format='user item rating timestamp', sep=',')
# 从文件读取数据
music_data = Dataset.load_from_file(file_path, reader=reader)
# 构建数据集和建模
algo = NMF()
trainset = music_data.build_full_trainset()
algo.train(trainset)

In [14]:
user_inner_id = 4
user_rating = trainset.ur[user_inner_id]
items = map(lambda x:x[0], user_rating)
for song in items:
    print(algo.predict(algo.trainset.to_raw_uid(user_inner_id), algo.trainset.to_raw_iid(song), r_ui=1), song_id_name_dic[algo.trainset.to_raw_iid(song)])

user: 400232387  item: 27724082   r_ui = 1.00   est = 1.00   {u'actual_k': 39, u'was_impossible': False} 听见下雨的声音	魏如昀
user: 400232387  item: 167916     r_ui = 1.00   est = 1.00   {u'actual_k': 40, u'was_impossible': False} 梦一场	萧敬腾
user: 400232387  item: 408307325  r_ui = 1.00   est = 1.00   {u'actual_k': 2, u'was_impossible': False} 干杯	西瓜Kune
user: 400232387  item: 394618     r_ui = 1.00   est = 1.00   {u'actual_k': 13, u'was_impossible': False} 给自己的歌 (Live) - live	纵贯线
user: 400232387  item: 421423806  r_ui = 1.00   est = 1.00   {u'actual_k': 40, u'was_impossible': False} 小半	陈粒
user: 400232387  item: 394485     r_ui = 1.00   est = 1.00   {u'actual_k': 33, u'was_impossible': False} 思念是一种病(Live) - live	张震岳
user: 400232387  item: 5239563    r_ui = 1.00   est = 1.00   {u'actual_k': 7, u'was_impossible': False} 可以不可以	丁当
user: 400232387  item: 30635613   r_ui = 1.00   est = 1.00   {u'actual_k': 9, u'was_impossible': False} 秋酿	房东的猫
user: 400232387  item: 185884     r_ui = 1.00   est = 1.00   {

In [15]:
#模型存储
import surprise
surprise.dump.dump('./recommendation.model', algo=algo)
# 可以用下面的方式载入
algo = surprise.dump.load('./recommendation.model')

In [16]:
import surprise
surprise.dump.dump('./recommendation.model', algo=algo)
# 可以用下面的方式载入
algo = surprise.dump.load('./recommendation.model')

## 不同的推荐系统算法评估

In [17]:
import os
from surprise import Reader, Dataset
# 指定文件路径
file_path = os.path.expanduser('./popular_music_suprise_format.txt')
# 指定文件格式
reader = Reader(line_format='user item rating timestamp', sep=',')
# 从文件读取数据
music_data = Dataset.load_from_file(file_path, reader=reader)
# 分成5折
music_data.split(n_folds=5)

In [18]:
music_data

<surprise.dataset.DatasetAutoFolds instance at 0x000000001410D308>

In [20]:
music_data.raw_ratings[:20]

[(u'418903732', u'29740084', 1.0, u'1300000'),
 (u'147893028', u'239026', 1.0, u'1300000'),
 (u'64382016', u'150617', 1.0, u'1300000'),
 (u'75787346', u'41664778', 1.0, u'1300000'),
 (u'99805197', u'28188304', 1.0, u'1300000'),
 (u'380792901', u'29818117', 1.0, u'1300000'),
 (u'11325305', u'95410', 1.0, u'1300000'),
 (u'379874207', u'35403523', 1.0, u'1300000'),
 (u'123294090', u'159929', 1.0, u'1300000'),
 (u'29804785', u'186055', 1.0, u'1300000'),
 (u'370390121', u'4873341', 1.0, u'1300000'),
 (u'324177726', u'185699', 1.0, u'1300000'),
 (u'89117406', u'382649', 1.0, u'1300000'),
 (u'312274808', u'142580', 1.0, u'1300000'),
 (u'35608485', u'187899', 1.0, u'1300000'),
 (u'376529297', u'254328', 1.0, u'1300000'),
 (u'156934569', u'29999898', 1.0, u'1300000'),
 (u'172907535', u'414979558', 1.0, u'1300000'),
 (u'11325305', u'104602', 1.0, u'1300000'),
 (u'85457463', u'229363', 1.0, u'1300000')]

In [22]:
### 使用NormalPredictor
from surprise import evaluate
# Evaluate performances of our algorithm on the dataset.
perf = evaluate(algo, music_data, measures=['RMSE', 'MAE'])
print_perf(perf)

Evaluating RMSE, MAE of algorithm tuple.

------------
Fold 1


AttributeError: 'tuple' object has no attribute 'fit'